# Load Features

In [1]:
import pickle
X_train = pickle.load(open( 'feat_table/xgb_1_x_train_table.pkl', "rb" ))
y_train = pickle.load(open( 'feat_table/xgb_1_y_train_table.pkl', "rb" ))
X_valid = pickle.load(open( 'feat_table/xgb_1_x_valid_table.pkl', "rb" ))
y_valid = pickle.load(open( 'feat_table/xgb_1_y_valid_table.pkl', "rb" ))
test = pickle.load(open( 'feat_table/xgb_1_test_table.pkl', "rb" ))
features = pickle.load(open( 'feat_table/xgb_1_features_vector_vector.pkl', "rb" ))

# LSTM Model w new loss function

Original simple model but with a similar loss function as the xgb

In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout
import numpy as np

Using TensorFlow backend.


### Model Specific Preprocessing

In [3]:
x_train_array = np.copy(X_train[features].values)
x_valid_array = np.copy(X_valid[features].values)

In [4]:
format_sub_train_x = x_train_array.reshape((x_train_array.shape[0], 1, x_train_array.shape[1]) )
format_sub_val_x = x_valid_array.reshape((x_valid_array.shape[0], 1, x_valid_array.shape[1]) )

### Define new loss Function

In [5]:
from keras import backend as K
def mean_squared_percentage_error(y_true, y_pred):
    return K.mean(K.square((y_true - y_pred) / K.clip(K.abs(y_true),
                                            K.epsilon(),
                                            None)), axis=-1)
mspe = MSPE = mean_squared_percentage_error

### Model Definition

In [6]:
model = Sequential()
model.add(LSTM(500, input_shape=(format_sub_train_x.shape[1], format_sub_train_x.shape[2]), 
               return_sequences = True) )
model.add(Dropout(0.2))
model.add(LSTM(500) )
model.add(Dense(1))

model.compile(loss=mean_squared_percentage_error, optimizer='adam')

# Train Model

Try a more fancy model rather than just the one layer from before.
Noticed that CUDA wasn't initialising properly

In [ ]:
mod_3 = model.fit(format_sub_train_x, np.copy(y_train), epochs=100, batch_size=100, 
                    validation_data=(format_sub_val_x, np.copy(y_valid)), verbose=2, shuffle=False)

### Save Model

In [12]:
pickle.dump(model, open( 'model/LSTM_Model_3.pkl', "wb" ))

PicklingError: Can't pickle <class 'module'>: attribute lookup module on builtins failed

# Prediction Code

In [13]:
dtest = np.copy(test[features].values)
test_frame = dtest.reshape((dtest.shape[0], 1, dtest.shape[1]) )

In [14]:
test_probs = model.predict(test_frame)
# Make Submission

In [15]:
import pandas as pd
result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs.flatten())})

In [16]:
result.to_csv("submission/lstm_model3_submission.csv", index=False)